In [5]:
# Import Packages
import pandas as pd
import numpy as np
from gurobipy import *
import matplotlib.pyplot as plt # Import Matplotlib tool kit 
import networkx as nx # Import NetworkX tool kit
from pulp import *
from scipy.optimize import linprog
from cvxopt import matrix
from cvxopt import glpk

from scipy.optimize import minimize
from matplotlib import cm
import cvxpy as cp
from scipy.linalg import circulant

In [ ]:
#------------------PROBLEM 4 MINIMUM COST NETWORK FLOW--------------------------------------------------------------------------
# The code is retrieved and referenced from below sources
#https://developers.google.com/optimization/flow/mincostflow
#https://machinelearninggeek.com/transshipment-problem-in-python-using-pulp/

In [9]:
#Transshipment Problem
#input values about routes and costs
#data =pd.read_excel(r'C:\Users\thaot\Desktop\HW 5 Excel input.xlsx',sheet_name='q4')
data =pd.read_excel("C:/Users/thanhthaopro/Downloads/hw5 excel for optimization.xlsx",sheet_name='Sheet1')
data.head()

In [ ]:
# Create lists of routes, costs, supply, demand and transhipment 
routes =data['routes']
shipping_cost = dict(zip(routes,data['cost']))

production_cost = dict(zip(routes,data['prod_cost']))


supply = {1:6500, 2:6000}
demand = {3:0,4:0,5:5000, 6:4000,7:3000,8:500}
transhipment = {3:3000, 4:4000}

In [ ]:
#Initialize LP Model
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Minimizing_the_cost",LpMinimize)
#variable declaration 
route_qty = LpVariable.dicts("arc",routes,lowBound=0,upBound=2200,cat='Continuous')

#Define Objective Function
# The objective function is added to 'prob' first
prob += lpSum([(shipping_cost[route]*route_qty[route] + production_cost[route]*route_qty[route])  for route in routes])

#enforcing the supply or manufacturing constraints
for  i in [1,2]:
    relevant = []
    for id,key in enumerate(route_qty.keys()):
        if not(key.find(str(i)) ==-1):
            relevant.append(key)
    print(relevant)
    print(supply[i])
    prob+=lpSum([route_qty[route] for route in relevant])==supply[i]

In [ ]:
#enforcing the demand constraints 
for  i in [5,6,7,8]:
    relevant = []
    for id,key in enumerate(route_qty.keys()):
        if not(key[-1].find(str(i)) ==-1):
            relevant.append(key)
    print(relevant)
    print(demand[i])
    print(lpSum([route_qty[route] for route in relevant]))
    prob+=lpSum([route_qty[route] for route in relevant])==demand[i]

In [ ]:
#enforcing flow balance at transhipment (in flow is equal to out flow)
for  i in [3,4]:
    relevant = []
    relevant_out = []
    for id,key in enumerate(route_qty.keys()):
        if not(key[-1].find(str(i)) ==-1):
            relevant.append(key)
        if key[-3]==str(i):
            relevant_out.append(key)

    print(lpSum(([route_qty[route] for route in relevant]) ) - lpSum(([route_qty[route] for route in relevant_out]) ))
    prob+=lpSum(([route_qty[route] for route in relevant]) ) - lpSum(([route_qty[route] for route in relevant_out]) )==demand[i]

In [ ]:
# solving the problem using pulp solver
prob.solve()
print(pulp.value(prob.objective))
# printing the status of the solution
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

In [ ]:
#------------4c adding new constraint for transhipment nodes Denver and Memphis------------------------------------------------

In [ ]:
#enforcing the demand constraints
for  i in [3,4]:
    relevant = []
    for id,key in enumerate(route_qty.keys()):
        if not(key[-1].find(str(i)) ==-1):
            relevant.append(key)
    print(relevant)
    
    print(lpSum([route_qty[route] for route in relevant]))
    prob+=lpSum([route_qty[route] for route in relevant])==transhipment[i]

# solving the problem using pulp solver
prob.solve()
print(pulp.value(prob.objective))
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

In [ ]:
#------------4d restrict number of cars can be shipped from Denver to LA and Memphis to Atlanta---------------------------------

In [ ]:
#enforcing the new capacity
relevant = ['route_3_5', 'route_4_6']    
print(lpSum([route_qty[route] for route in relevant]))
prob+=lpSum([route_qty[route] for route in relevant])<=3000

# solving the problem using pulp solver
prob.solve()
print(pulp.value(prob.objective))
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)